<a href="https://colab.research.google.com/github/takumi-maker/bert/blob/main/momentum_factor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import math
import statistics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression

In [ ]:
df_1 = pd.read_csv("債券複利データセット3.csv")
df_1 = df_1.drop(df_1.columns[0], axis=1)
columns_len = len(df_1.columns)
length = len(df_1)
df_1
y_df = df_1["GJGC10"]
X_df = df_1.drop("GJGC10", axis=1)
y_df

,GJGC10
0,0.030
1,0.025
2,0.030
3,0.030
4,0.045
...,...
974,1.114
975,1.114
976,1.176
977,1.172


In [ ]:
df_2 = pd.read_csv("債券複利データセット3.csv")
df_2 = df_2.drop(df_2.columns[0], axis=1)
columns_len = len(df_2.columns)
length = len(df_2)
df_2
y_df2 = df_2["GJGC20"]
X_df2 = df_2.drop("GJGC20", axis=1)
y_df2

,GJGC20
0,0.414
1,0.405
2,0.410
3,0.414
4,0.438
...,...
974,1.898
975,1.900
976,1.934
977,1.918


In [ ]:
y_df = pd.concat([y_df, y_df2], axis=1)
y_df["select_spread"] = y_df["GJGC20"] - y_df["GJGC10"]
y_df = y_df.drop('GJGC20',axis=1)
y_df = y_df.drop('GJGC10',axis=1)

In [ ]:
y_df

,select_spread
0,0.384
1,0.380
2,0.380
3,0.384
4,0.393
...,...
974,0.784
975,0.786
976,0.758
977,0.746


In [ ]:
from sklearn.linear_model import LinearRegression
import statsmodels
import statsmodels.api
import matplotlib.pyplot as plt
from sklearn import preprocessing
import cvxpy as cp

magic_num = 30
estimate_num = 150

df_array = y_df.to_numpy()

for i in range(1,8):
  total_num = 100*i
  print(total_num)

  total_list = []
  y_list = []
  result_list = []
  predict_flag2 = 0
  for j in range(magic_num-2,len(df_array)):
    X_return_list = []
    for k in range(1,magic_num):
      X_return_list.append((df_array[j-k]-df_array[j-k-1])*10)
    y_return = (df_array[j]-df_array[j-1])*10
    y_list.append(y_return)
    X_list2 = list(reversed(X_return_list))
    total_list.append(X_list2)
  #print(len(total_list))

  for j in range(len(total_list)):
    if j <= len(total_list)-1-total_num:
      continue
    #print(j)
    X_array = np.squeeze(np.array(total_list[j]))
    y_array = y_list[j][0]
    estimate_list = []
    estimate_y_list = []
    for k in range(1,estimate_num):
      estimate_list.append(total_list[j-k])
      estimate_y_list.append(y_list[j-k][0])

    estimate_list = np.array(estimate_list)
    estimate_list = np.squeeze(estimate_list)
    estimate_y_list = np.array(estimate_y_list)
    #print(estimate_list)
    #print(estimate_y_list)


    #alpha = np.array([0,0.1,0.2,0.3,0.4,0.5])
    #print(estimate_list.shape)
    #print(estimate_y_list.shape)
    k= estimate_list.shape[1]
    beta = cp.Variable(k,nonneg=True)
    constraints =[beta[i] <= beta[i+1] for i in range(k-1)]
    #error = cp.norm2(estimate_y_list - estimate_list @ beta)
    #objective =  cp.sum_squares(estimate_y_list - estimate_list @ beta)
    objective =  cp.norm2(estimate_y_list - estimate_list @ beta)
    #objective = cp.Minimize(error)


    #prob = cp.Problem(objective, constraints)
    prob = cp.Problem(cp.Minimize(objective), constraints)
    prob.solve()
    #print("status:", prob.status)
    #print("optimal value", prob.value)
    #print("optimal var", beta.value)

    predict = np.dot(X_array,beta.value)
    #print(predict)
    #print(y_array)

    if predict >= 0:
      predict_flag = -1
    else:
      predict_flag = 1

    if predict_flag*predict_flag2 >=0:
      result_r = y_array*predict_flag
      result_list.append(result_r)
      predict_flag2 = predict_flag
    else:
      result_r = y_array*predict_flag-0.005
      result_list.append(result_r)
      predict_flag2 = predict_flag

  R_sum = sum(result_list)
  SR = np.mean(np.array(result_list))/np.std(np.array(result_list))
  print("SharpRatio:",SR)
  print(R_sum)




100
SharpRatio: 0.11959279876234526
1.3549999999999898
200
SharpRatio: 0.006884893195198731
0.15999999999998804
300
SharpRatio: 0.015995101839033108
0.5849999999999809
400
SharpRatio: 0.05137617266918158
2.545000000000002
500
SharpRatio: 0.061309482134364976
5.094999999999997
600
SharpRatio: 0.04418351258800453
5.110000000000008
700
SharpRatio: 0.03047155321415066
4.075000000000021


In [ ]:
from sklearn.linear_model import LinearRegression
import statsmodels
import statsmodels.api
import matplotlib.pyplot as plt
from sklearn import preprocessing
import cvxpy as cp

magic_num = 30
estimate_num = 150

df_array = y_df2.to_numpy()

for i in range(1,8):
  total_num = 100*i
  print(total_num)

  total_list = []
  y_list = []
  result_list = []
  predict_flag2 = 0
  for j in range(magic_num-2,len(df_array)):
    X_return_list = []
    for k in range(1,magic_num):
      X_return_list.append((df_array[j-k]-df_array[j-k-1])*10)
    y_return = (df_array[j]-df_array[j-1])*10
    y_list.append(y_return)
    X_list2 = list(reversed(X_return_list))
    total_list.append(X_list2)
  #print(len(total_list))

  for j in range(len(total_list)):
    if j <= len(total_list)-1-total_num:
      continue
    #print(j)
    X_array = np.squeeze(np.array(total_list[j]))
    y_array = y_list[j]
    estimate_list = []
    estimate_y_list = []
    for k in range(1,estimate_num):
      estimate_list.append(total_list[j-k])
      estimate_y_list.append(y_list[j-k])

    estimate_list = np.array(estimate_list)
    estimate_list = np.squeeze(estimate_list)
    estimate_y_list = np.array(estimate_y_list)
    #print(estimate_list)
    #print(estimate_y_list)


    #alpha = np.array([0,0.1,0.2,0.3,0.4,0.5])
    #print(estimate_list.shape)
    #print(estimate_y_list.shape)
    k= estimate_list.shape[1]
    beta = cp.Variable(k,nonneg=True)
    constraints =[beta[i] <= beta[i+1] for i in range(k-1)]
    #error = cp.norm2(estimate_y_list - estimate_list @ beta)
    objective =  cp.sum_squares(estimate_y_list - estimate_list @ beta)
    #objective = cp.Minimize(error)


    #prob = cp.Problem(objective, constraints)
    prob = cp.Problem(cp.Minimize(objective), constraints)
    prob.solve()
    #print("status:", prob.status)
    #print("optimal value", prob.value)
    #print("optimal var", beta.value)

    predict = np.dot(X_array,beta.value)
    #print(predict)
    #print(y_array)

    if predict >= 0:
      predict_flag = -1
    else:
      predict_flag = 1

    if predict_flag*predict_flag2 >=0:
      result_r = y_array*predict_flag
      result_list.append(result_r)
      predict_flag2 = predict_flag
    else:
      result_r = y_array*predict_flag-0.005
      result_list.append(result_r)
      predict_flag2 = predict_flag

  R_sum = sum(result_list)
  SR = np.mean(np.array(result_list))/np.std(np.array(result_list))
  print("SharpRatio:",SR)
  print(R_sum)



100


KeyboardInterrupt: 

In [ ]:
!pip install cvxpy